In [ ]:
#!pip install transformers datasets seqeval -q

#!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d5467df00b8fc1984217f43fff51c7b58dd583a4e34b1fbb362580cd82f2018c
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch


In [ ]:
from google.colab import files
uploaded = files.upload()

#df = load_conll_file("/content/ner_train.conll")

Saving ner_train.conll to ner_train.conll


#Load and Parse CoNLL File

In [ ]:
def load_conll_file(path):
    tokens, labels = [], []
    all_tokens, all_labels = [], []

    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip() == "":
                if tokens:
                    all_tokens.append(tokens)
                    all_labels.append(labels)
                    tokens, labels = [], []
            else:
                token, label = line.strip().split()
                tokens.append(token)
                labels.append(label)

    return pd.DataFrame({'tokens': all_tokens, 'ner_tags': all_labels})

df = load_conll_file("ner_train.conll")
df.head()


,tokens,ner_tags
0,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
1,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
2,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
3,"[TIMBERLAND, Made, In, VIETNAM, Size, #40, #41...","[B-Product, O, O, O, O, O, O, O, O, O, O, B-PR..."


#Prepare dataset

In [ ]:
split = df.sample(frac=1, random_state=42)
train_df = split[:int(0.8 * len(split))]
val_df = split[int(0.8 * len(split)):]

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

dataset = DatasetDict({"train": train_ds, "validation": val_ds})
df.head()


,tokens,ner_tags
0,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
1,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
2,"[CLASSY, BRAND, Baggy, Pant, Price, 2999br, ለሱ...","[O, O, B-Product, B-Product, O, B-PRICE, O, O,..."
3,"[TIMBERLAND, Made, In, VIETNAM, Size, #40, #41...","[B-Product, O, O, O, O, O, O, O, O, O, O, B-PR..."


#Tokenizer & labels

In [ ]:
label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'CONTACT_INFO']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(example):
    tokenized = tokenizer(example["tokens"], is_split_into_words=True, padding='max_length', truncation=True, max_length=128)
    word_ids = tokenized.word_ids()
    labels = []
    for i in word_ids:
        if i is None:
            labels.append(-100)
        else:
            labels.append(label_to_id[example["ner_tags"][i]])
    tokenized["labels"] = labels
    return tokenized

tokenized_dataset = dataset.map(tokenize_and_align_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

#Load model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Training setup

In [ ]:
args = TrainingArguments(
    output_dir="ner-xlm-roberta",
    #eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    preds, labels = p.predictions.argmax(-1), p.label_ids
    true_preds = [[id_to_label[p] for (p, l) in zip(pred, lab) if l != -100] for pred, lab in zip(preds, labels)]
    true_labels = [[id_to_label[l] for (p, l) in zip(pred, lab) if l != -100] for pred, lab in zip(preds, labels)]
    return metric.compute(predictions=true_preds, references=true_labels)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


#Train and save model

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable W&B BEFORE Trainer init

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

os.environ["WANDB_DISABLED"] = "true"
trainer.train()
#save
model.save_pretrained("xlm-roberta-amharic-ner")
tokenizer.save_pretrained("xlm-roberta-amharic-ner")



/tmp/ipython-input-19-3556361282.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


('xlm-roberta-amharic-ner/tokenizer_config.json',
 'xlm-roberta-amharic-ner/special_tokens_map.json',
 'xlm-roberta-amharic-ner/sentencepiece.bpe.model',
 'xlm-roberta-amharic-ner/added_tokens.json',
 'xlm-roberta-amharic-ner/tokenizer.json')

zip and download model

In [7]:
# !git config --global user.email "tnsaydagne@gmail.com"
# !git config --global user.name "tnsay"

#!git clone https://github.com/tnsay/ethioMart-telegram-ner.git
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/xlm-roberta-amharic-ner /content/drive/MyDrive/ethioMart-telegram-ner/xlm-roberta-amharic-ner
!cp /content/drive/MyDrive/Colab_Notebooks/Fine-TuneEcomTask3.ipynb ethioMart-telegram-ner/FineTuneNER_Task3.ipynb



In [ ]:
!zip -r model.zip xlm-roberta-amharic-ner
from google.colab import files
#files.download("model.zip")

	zip warning: name not matched: xlm-roberta-amharic-ner

zip error: Nothing to do! (try: zip -r model.zip . -i xlm-roberta-amharic-ner)


FileNotFoundError: Cannot find file: model.zip